In [6]:
# Install required libraries
!pip install transformers datasets sklearn
!pip  install Datasets
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Load and prepare the dataset
file_path = 'Tesla_Trustpilot_Reviews.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Prepare the dataset for Hugging Face
def prepare_data(df):
    df = df[['Content', 'Rating']].dropna()  # Use Content and Rating columns
    df['label'] = df['Rating'].apply(lambda x: int(x) - 1)  # Convert ratings (1-5) to labels (0-4)
    return df

df = prepare_data(df)

# Split into training and testing datasets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the dataset
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["Content"], truncation=True, padding="max_length", max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
print("Training the model...")
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("Model training completed and saved.")

# Perform sentiment predictions
print("Performing sentiment predictions...")
fine_tuned_model = pipeline(
    "text-classification",
    model="./fine_tuned_model",
    tokenizer="./fine_tuned_model"
)

# Apply predictions to the dataset
df['emotion_label'] = df['Content'].apply(lambda x: fine_tuned_model(x[:512])[0]['label'])
df['emotion_prob'] = df['Content'].apply(lambda x: fine_tuned_model(x[:512])[0]['score'])

# Save the updated dataset
output_file = 'Tesla_Trustpilot_Reviews_with_FineTuned_Sentiment.csv'
df.to_csv(output_file, index=False)
print(f"Sentiment analysis completed. Results saved to '{output_file}'.")


Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'sci

Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------- ----------- 41.0/57.7 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/480.6 kB ? eta -:--:--
   --- ------------------------------------ 41.0/480.6 kB 2.0 MB/s eta 0:00:01
   ---------------- ----------------------- 194.6/480.6 kB 2.4 MB/s eta 0:00:01
   ----------------------- ---------------- 276.5/480.6 kB 2.9 MB/s eta 0:00:01
   ---------------------------------- ----- 409.6/480.6 kB 2.6 MB/s eta 0:00:01
   ------------------------------------ --- 440.3/480.6 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 480.6/480.6 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   --

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\moham\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\moham\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/h

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.
